In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a FAISS VectorStore
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings() # deployment="text-embedding-ada-002"
)

# Create a Retriever
retriever = vectorstore.as_retriever()

# Create a prompt, model
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI() # model_name='gpt-3.5-turbo'

# Chaining a pipeline
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Run retrieva_chain
retrieval_chain.invoke("where did harrison work?")
# 'Harrison worked at Kensho.'


In [13]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="llama3.3", base_url="172.17.0.2:11434")

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = FAISS.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

In [10]:
retriever.invoke("Hello?")

[Document(id='bb8ecfce-8fe7-4b1b-bce7-e7f34a58f046', metadata={}, page_content='LangChain is the framework for building context-aware reasoning applications')]

In [14]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

In [15]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [16]:
print(example_prompt.invoke(examples[0]).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [18]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

In [21]:
# from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OllamaEmbeddings(model="llama3.3", base_url="172.17.0.2:11434"),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


question: Who was the maternal grandfather of George Washington?
answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball



In [22]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


In [24]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3.3", model_provider="ollama", base_url="172.17.0.2:11434")


In [25]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'model': 'llama3.3', 'created_at': '2025-03-14T01:01:40.691581102Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13545661952, 'load_duration': 13184839170, 'prompt_eval_count': 24, 'prompt_eval_duration': 159648000, 'eval_count': 4, 'eval_duration': 155859000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-dbcd79b5-37e7-4f6c-beb2-1764011feb09-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

In [28]:
model.invoke([
    {"role": "system", "content": "Translate the following from English into Korean, very friendly like friend."},
    {"role": "user", "content": "Hello"}
    ])



AIMessage(content='안녕!! 어떻게 지내고 있어?', additional_kwargs={}, response_metadata={'model': 'llama3.3', 'created_at': '2025-03-14T01:04:03.126799172Z', 'done': True, 'done_reason': 'stop', 'total_duration': 755716457, 'load_duration': 21647917, 'prompt_eval_count': 29, 'prompt_eval_duration': 108504000, 'eval_count': 10, 'eval_duration': 495500000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-43b006f9-a038-46bf-b6b7-5c4fcb47c04c-0', usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39})

In [36]:
for token in model.stream([
    {"role": "system", "content": "Translate the following from English into Korean, very friendly like friend."},
    {"role": "user", "content": "Hello"}
    ]):
    print(token.content, end="")

안녕!!

In [29]:
for token in model.stream(messages):
    print(token.content, end="|")

C|iao|!||

In [40]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({"language": "Korean", "text": "hi!"})

prompt
response = model.invoke(prompt)
print(response.content)

안녕!


In [41]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [45]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./manual.md")
documents = loader.load()

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(documents)

len(all_splits)


70

In [50]:
embeddings = OllamaEmbeddings(model="llama3.3", base_url="172.17.0.2:11434")
v1 = embeddings.embed_query(all_splits[0].page_content)
v2 = embeddings.embed_query(all_splits[1].page_content)
assert len(v1) == len(v2)
print(f"Generated vectors of length {len(v1)}\n")
print(v1[:10])

# text = "LangChain is the framework for building context-aware reasoning applications"

# vectorstore = FAISS.from_texts(
#     [text],
#     embedding=embeddings,
# )

# # Use the vectorstore as a retriever
# retriever = vectorstore.as_retriever()

# # Retrieve the most similar text
# retrieved_documents = retriever.invoke("What is LangChain?")

# # show the retrieved document's content
# retrieved_documents[0].page_content

Generated vectors of length 8192

[0.002270376, -0.0002960831, -0.0005981432, 0.0067523727, -0.0017729027, -0.008718284, -0.010643234, -0.00091156556, 0.00011833437, -0.01181567]


In [54]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vectorstore = FAISS(embedding_function=embeddings, index=index, docstore=InMemoryDocstore(), index_to_docstore_id={})


In [55]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vectorstore.add_documents(documents=documents, ids=uuids)

['fc4aa937-32c9-4665-9b93-b24db2031bb4',
 '4557935f-7586-4e1d-bfd3-d106dcd7a389',
 'b5ec9ad0-8a82-4147-a007-2f64547077e4',
 '9b304f47-3c9e-42b4-90c2-431705ff4d60',
 'a9ebfa69-68ea-4f72-96f2-dd948929072a',
 '8c17ee95-628f-4df6-8220-d8d7ae7affeb',
 '67bbb46f-25ba-415b-857e-008b55f55a55',
 '20539fa1-9e52-498d-a914-a89aaac4cc56',
 '1baf27e7-05df-45c7-a23d-39ec82d4c14a',
 '58bdaef0-9e18-46b3-8e54-68abf56fa9ae']

In [59]:
results = vectorstore.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]


In [61]:
results = vectorstore.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.553510] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [62]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='1baf27e7-05df-45c7-a23d-39ec82d4c14a', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

In [63]:
vectorstore.save_local("faiss_index")

In [64]:
new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [65]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

RANDOM_SEED = 42  # 재현성을 위한 고정된 시드 값

### --- 위의 인용된 코드에서 주석과 문서화를 추가함 --- ###


def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    UMAP을 사용하여 임베딩의 전역 차원 축소를 수행합니다.

    매개변수:
    - embeddings: numpy 배열로 된 입력 임베딩.
    - dim: 축소된 공간의 목표 차원.
    - n_neighbors: 선택 사항; 각 점을 고려할 이웃의 수.
                   제공되지 않으면 임베딩 수의 제곱근으로 기본 설정됩니다.
    - metric: UMAP에 사용할 거리 측정 기준.

    반환값:
    - 지정된 차원으로 축소된 임베딩의 numpy 배열.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


In [66]:
def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    임베딩에 대해 지역 차원 축소를 수행합니다. 이는 일반적으로 전역 클러스터링 이후에 사용됩니다.

    매개변수:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - dim: 축소된 공간의 목표 차원 수.
    - num_neighbors: 각 점에 대해 고려할 이웃의 수.
    - metric: UMAP에 사용할 거리 측정 기준.

    반환값:
    - 지정된 차원으로 축소된 임베딩의 numpy 배열.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


In [67]:
def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    가우시안 혼합 모델(Gaussian Mixture Model)을 사용하여 베이지안 정보 기준(BIC)을 통해 최적의 클러스터 수를 결정합니다.

    매개변수:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - max_clusters: 고려할 최대 클러스터 수.
    - random_state: 재현성을 위한 시드.

    반환값:
    - 발견된 최적의 클러스터 수를 나타내는 정수.
    """
    max_clusters = min(
        max_clusters, len(embeddings)
    )  # 최대 클러스터 수와 임베딩의 길이 중 작은 값을 최대 클러스터 수로 설정
    n_clusters = np.arange(1, max_clusters)  # 1부터 최대 클러스터 수까지의 범위를 생성
    bics = []  # BIC 점수를 저장할 리스트
    for n in n_clusters:  # 각 클러스터 수에 대해 반복
        gm = GaussianMixture(
            n_components=n, random_state=random_state
        )  # 가우시안 혼합 모델 초기화
        gm.fit(embeddings)  # 임베딩에 대해 모델 학습
        bics.append(gm.bic(embeddings))  # 학습된 모델의 BIC 점수를 리스트에 추가
    return n_clusters[np.argmin(bics)]  # BIC 점수가 가장 낮은 클러스터 수를 반환


In [68]:
def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    확률 임계값을 기반으로 가우시안 혼합 모델(GMM)을 사용하여 임베딩을 클러스터링합니다.

    매개변수:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - threshold: 임베딩을 클러스터에 할당하기 위한 확률 임계값.
    - random_state: 재현성을 위한 시드.

    반환값:
    - 클러스터 레이블과 결정된 클러스터 수를 포함하는 튜플.
    """
    n_clusters = get_optimal_clusters(embeddings)  # 최적의 클러스터 수를 구합니다.
    # 가우시안 혼합 모델을 초기화합니다.
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)  # 임베딩에 대해 모델을 학습합니다.
    probs = gm.predict_proba(
        embeddings
    )  # 임베딩이 각 클러스터에 속할 확률을 예측합니다.
    # 임계값을 초과하는 확률을 가진 클러스터를 레이블로 선택합니다.
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters  # 레이블과 클러스터 수를 반환합니다.


In [69]:
def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    임베딩에 대해 차원 축소, 가우시안 혼합 모델을 사용한 클러스터링, 각 글로벌 클러스터 내에서의 로컬 클러스터링을 순서대로 수행합니다.

    매개변수:
    - embeddings: numpy 배열로 된 입력 임베딩입니다.
    - dim: UMAP 축소를 위한 목표 차원입니다.
    - threshold: GMM에서 임베딩을 클러스터에 할당하기 위한 확률 임계값입니다.

    반환값:
    - 각 임베딩의 클러스터 ID를 포함하는 numpy 배열의 리스트입니다.
    """
    if len(embeddings) <= dim + 1:
        # 데이터가 충분하지 않을 때 클러스터링을 피합니다.
        return [np.array([0]) for _ in range(len(embeddings))]

    # 글로벌 차원 축소
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # 글로벌 클러스터링
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # 각 글로벌 클러스터를 순회하며 로컬 클러스터링 수행
    for i in range(n_global_clusters):
        # 현재 글로벌 클러스터에 속하는 임베딩 추출
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # 작은 클러스터는 직접 할당으로 처리
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # 로컬 차원 축소 및 클러스터링
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # 로컬 클러스터 ID 할당, 이미 처리된 총 클러스터 수를 조정
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


In [70]:
def embed(texts):
    # 텍스트 문서 목록에 대한 임베딩을 생성합니다.
    #
    # 이 함수는 `embd` 객체가 존재한다고 가정하며, 이 객체는 텍스트 목록을 받아 그 임베딩을 반환하는 `embed_documents` 메소드를 가지고 있습니다.
    #
    # 매개변수:
    # - texts: List[str], 임베딩할 텍스트 문서의 목록입니다.
    #
    # 반환값:
    # - numpy.ndarray: 주어진 텍스트 문서들에 대한 임베딩 배열입니다.
    text_embeddings = embeddings.embed_documents(
        texts
    )  # 텍스트 문서들의 임베딩을 생성합니다.
    text_embeddings_np = np.array(text_embeddings)  # 임베딩을 numpy 배열로 변환합니다.
    return text_embeddings_np  # 임베딩된 numpy 배열을 반환합니다.


In [71]:
def embed_cluster_texts(texts):
    """
    텍스트 목록을 임베딩하고 클러스터링하여, 텍스트, 그들의 임베딩, 그리고 클러스터 라벨이 포함된 DataFrame을 반환합니다.

    이 함수는 임베딩 생성과 클러스터링을 단일 단계로 결합합니다. 임베딩에 대해 클러스터링을 수행하는 `perform_clustering` 함수의 사전 정의된 존재를 가정합니다.

    매개변수:
    - texts: List[str], 처리될 텍스트 문서의 목록입니다.

    반환값:
    - pandas.DataFrame: 원본 텍스트, 그들의 임베딩, 그리고 할당된 클러스터 라벨이 포함된 DataFrame입니다.
    """
    text_embeddings_np = embed(texts)  # 임베딩 생성
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # 임베딩에 대해 클러스터링 수행
    df = pd.DataFrame()  # 결과를 저장할 DataFrame 초기화
    df["text"] = texts  # 원본 텍스트 저장
    df["embd"] = list(text_embeddings_np)  # DataFrame에 리스트로 임베딩 저장
    df["cluster"] = cluster_labels  # 클러스터 라벨 저장
    return df


In [72]:
def fmt_txt(df: pd.DataFrame) -> str:
    """
    DataFrame에 있는 텍스트 문서를 단일 문자열로 포맷합니다.

    매개변수:
    - df: 'text' 열에 포맷할 텍스트 문서가 포함된 DataFrame.

    반환값:
    - 모든 텍스트 문서가 특정 구분자로 결합된 단일 문자열.
    """
    unique_txt = df["text"].tolist()  # 'text' 열의 모든 텍스트를 리스트로 변환
    return "--- --- \n --- --- ".join(
        unique_txt
    )  # 텍스트 문서들을 특정 구분자로 결합하여 반환


In [73]:
def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    텍스트 목록에 대해 임베딩, 클러스터링 및 요약을 수행합니다. 이 함수는 먼저 텍스트에 대한 임베딩을 생성하고,
    유사성을 기반으로 클러스터링을 수행한 다음, 클러스터 할당을 확장하여 처리를 용이하게 하고 각 클러스터 내의 내용을 요약합니다.

    매개변수:
    - texts: 처리할 텍스트 문서 목록입니다.
    - level: 처리의 깊이나 세부 사항을 정의할 수 있는 정수 매개변수입니다.

    반환값:
    - 두 개의 데이터프레임을 포함하는 튜플:
      1. 첫 번째 데이터프레임(`df_clusters`)은 원본 텍스트, 그들의 임베딩, 그리고 클러스터 할당을 포함합니다.
      2. 두 번째 데이터프레임(`df_summary`)은 각 클러스터에 대한 요약, 지정된 세부 수준, 그리고 클러스터 식별자를 포함합니다.
    """

    # 텍스트를 임베딩하고 클러스터링하여 'text', 'embd', 'cluster' 열이 있는 데이터프레임을 생성합니다.
    df_clusters = embed_cluster_texts(texts)

    # 클러스터를 쉽게 조작하기 위해 데이터프레임을 확장할 준비를 합니다.
    expanded_list = []

    # 데이터프레임 항목을 문서-클러스터 쌍으로 확장하여 처리를 간단하게 합니다.
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # 확장된 목록에서 새 데이터프레임을 생성합니다.
    expanded_df = pd.DataFrame(expanded_list)

    # 처리를 위해 고유한 클러스터 식별자를 검색합니다.
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # 요약
    template = """여기 LangChain 표현 언어 문서의 하위 집합이 있습니다.

    LangChain 표현 언어는 LangChain에서 체인을 구성하는 방법을 제공합니다.

    제공된 문서의 자세한 요약을 제공하십시오.

    문서:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()

    # 각 클러스터 내의 텍스트를 요약을 위해 포맷팅합니다.
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # 요약, 해당 클러스터 및 레벨을 저장할 데이터프레임을 생성합니다.
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


In [74]:
def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    지정된 레벨까지 또는 고유 클러스터의 수가 1이 될 때까지 텍스트를 재귀적으로 임베딩, 클러스터링, 요약하여
    각 레벨에서의 결과를 저장합니다.

    매개변수:
    - texts: List[str], 처리할 텍스트들.
    - level: int, 현재 재귀 레벨 (1에서 시작).
    - n_levels: int, 재귀의 최대 깊이.

    반환값:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], 재귀 레벨을 키로 하고 해당 레벨에서의 클러스터 DataFrame과 요약 DataFrame을 포함하는 튜플을 값으로 하는 사전.
    """
    results = {}  # 각 레벨에서의 결과를 저장할 사전

    # 현재 레벨에 대해 임베딩, 클러스터링, 요약 수행
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # 현재 레벨의 결과 저장
    results[level] = (df_clusters, df_summary)

    # 추가 재귀가 가능하고 의미가 있는지 결정
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # 다음 레벨의 재귀 입력 텍스트로 요약 사용
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # 다음 레벨의 결과를 현재 결과 사전에 병합
        results.update(next_level_results)

    return results


In [85]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./manual.md")
documents = loader.load()

In [86]:
docs_texts = [d.page_content for d in documents]

In [87]:
docs_texts

['# BIFE-Cycler 설치/운영 매뉴얼\n\n어플리케이션의 설치를 위해서 필요한 인프라 소프트웨어/서버를 설치합니다. 다음과 같은 순서로 설치합니다.\n\n참고로 운영서버는 다음과 같습니다.\n\nSignet Production Servers\n=======================\n\n```\n| Server      | Inner IP    |  Port   | OS Version         |\n|-------------|-------------|---------|--------------------|\n| server-1    | 192.168.0.11|  43011  | Ubuntu 24.04.1 LTS |\n| server-2    | 192.168.0.12|  43012  | Ubuntu 24.04.1 LTS |\n| server-3    | 192.168.0.13|  43013  | Ubuntu 24.04.1 LTS |\n| server-4    | 192.168.0.14|  43014  | Ubuntu 24.04.1 LTS |\n```\n\n설치 순서는 크게 6개의 단계로 나누어 집니다.\n\n- 환경준비\n- 도커 설치\n- DB 설치\n- 카프카설치\n- 어플리케이션 설치\n- 카프카 클라이언트(데이터 수집) 설치\n\n\n각 서버는 /(root)에 /data directory 를 다음과 같이 가지고 있습니다.\n```bash \n/data\n├── config # 설정화일\n│   └── kafka\n│       └── server.properties\n├── db # postgresql data 저장소\n│   └── pgdata\n│       ├── conf\n│       │   ├── pg_hba.conf\n│       │   └── postgresql.conf\n│       ├── data  [error opening dir]\n│       └── lock\n│           └── master.loc

In [88]:
# 트리 구축
leaf_texts = docs_texts  # 문서 텍스트를 리프 텍스트로 설정
results = recursive_embed_cluster_summarize(
    leaf_texts, level=1, n_levels=3
)  # 재귀적으로 임베딩, 클러스터링 및 요약을 수행하여 결과를 얻음


--Generated 1 clusters--


In [84]:
results

{1: (                                                text  \
  0  # BIFE-Cycler 설치/운영 매뉴얼\n\n어플리케이션의 설치를 위해서 필요한...   
  
                                                  embd cluster  
  0  [0.004138216, 0.0005472095, 0.008901267, 0.003...     [0]  ,
                                             summaries  level  cluster
  0  It seems like you're experiencing some issues ...      1        0)}

In [89]:
from langchain_community.vectorstores import FAISS

# leaf_texts를 복사하여 all_texts를 초기화합니다.
all_texts = leaf_texts.copy()

# 각 레벨의 요약을 추출하여 all_texts에 추가하기 위해 결과를 순회합니다.
for level in sorted(results.keys()):
    # 현재 레벨의 DataFrame에서 요약을 추출합니다.
    summaries = results[level][1]["summaries"].tolist()
    # 현재 레벨의 요약을 all_texts에 추가합니다.
    all_texts.extend(summaries)

# 이제 all_texts를 사용하여 FAISS vectorstore를 구축합니다.
vectorstore = FAISS.from_texts(texts=all_texts, embedding=embeddings)


In [90]:
import os

DB_INDEX = "RAPTOR"

# 로컬에 FAISS DB 인덱스가 이미 존재하는지 확인하고, 그렇다면 로드하여 vectorstore와 병합한 후 저장합니다.
if os.path.exists(DB_INDEX):
    local_index = FAISS.load_local(DB_INDEX, embeddings)
    local_index.merge_from(vectorstore)
    local_index.save_local(DB_INDEX)
else:
    vectorstore.save_local(folder_path=DB_INDEX)


In [91]:
# retriever 생성
retriever = vectorstore.as_retriever()

In [92]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 생성
prompt = hub.pull("rlm/rag-prompt")

# 문서 포스트 프로세싱


def format_docs(docs):
    # 문서의 페이지 내용을 이어붙여 반환합니다.
    return "\n\n".join(doc.page_content for doc in docs)


# RAG 체인 정의
rag_chain = (
    # 검색 결과를 포맷팅하고 질문을 처리합니다.
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 적용합니다.
    | model  # 모델을 적용합니다.
    | StrOutputParser()  # 문자열 출력 파서를 적용합니다.
)


/usr/local/lib/python3.12/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [96]:
# 추상적인 질문 실행
rag_chain.invoke("show servers for bife-cycler application")


'Docker 및 Kafka 관련 문제 해결에 대한 가이드입니다.\n\n1. **도커 컨테이너 이름 충돌**:\n   - 도커 컨테이너를 실행할 때 이미 사용중인 이름으로 인해 오류가 발생하는 경우, 해당 컨테이너를 제거하거나 이름을 변경해야 합니다.\n   - 제거 방법: `docker rm <컨테이너 이름>`\n\n2. **웹 브라우저 접속 문제**:\n   - 로그 확인: `docker logs -f -n 1000 <인스턴스 이름>` 명령어를 사용하여 최근 로그를 확인합니다.\n   - 로그 해석이 어려울 경우, 캡처본을 개발사에 전달합니다.\n\n3. **데이터가 들어오지 않는 경우**:\n   - `kafka-subscriber`의 로그를 확인하여 데이터가 정상적으로 들어오는지 확인합니다.\n   - `docker ps -a | grep kafka-subscriber` 명령어로 `kafka-subscriber` 컨테이너가 실행 중인지 확인합니다.\n   - 재시동: `docker restart kafka-subscriber`\n\n4. **서버별 실행 스크립트**:\n   - Server 1, Server 2, Server 3 각각의 설치 디렉토리 내에서 다양한 스크립트를 실행할 수 있습니다. \n     - 예시: 도커 설치, PostgreSQL 실행, 카프카 실행, 애플리케이션 실행, 카프카 서브스크라이버 실행 등.\n\n이 가이드는 Docker와 Kafka 관련 문제 해결을 위한 기본적인 정보를 제공합니다. 자세한 내용이나 추가적인 도움이 필요할 경우 개발사나 전문가에게 상담하는 것이 좋습니다.'

In [109]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_ollama import OllamaLLM
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.vectorstores import FAISS
import faiss
llm = OllamaLLM(model="llama3.3")

document_content_description = "A guide to install and operation of a bife-cycler application."
metadata_field_info = [
    AttributeInfo(
        name="manual",
        description="The manual of the bife-cycler application",
        type="string",
    ),
    # Additional metadata fields...
]

retriever = vectorstore.as_retriever()


In [110]:
retriever.invoke("How to install bife-cycler?")

[Document(id='5879a589-5f7e-4780-83f8-7cbfd7648e43', metadata={}, page_content='Docker 및 카프카(Kafka) 관련된 문제 해결을 위한 가이드입니다. 주요 내용은 다음과 같습니다.\n\n1. **도커 컨테이너 이름 충돌**:\n   - 도커 컨테이너를 실행할 때 이미 사용중인 이름으로 인해 오류가 발생하는 경우, 해당 컨테이너를 제거하거나 이름을 변경해야 합니다.\n   - 제거 방법: `docker rm <컨테이너 이름>`\n\n2. **웹 브라우저 접속 문제**:\n   - 로그 확인: `docker logs -f -n 1000 <인스턴스 이름>` 명령어를 사용하여 최근 로그를 확인합니다.\n   - 로그 해석이 어려울 경우, 캡처본을 개발사에 전달합니다.\n\n3. **데이터가 들어오지 않는 경우**:\n   - `kafka-subscriber`의 로그를 확인하여 데이터가 정상적으로 들어오는지 확인합니다.\n   - `docker ps -a | grep kafka-subscriber` 명령어로 `kafka-subscriber` 컨테이너가 실행 중인지 확인합니다.\n   - 재시동: `docker restart kafka-subscriber`\n\n4. **서버별 실행 스크립트**:\n   - Server 1, Server 2, Server 3 각각의 설치 디렉토리 내에서 다양한 스크립트를 실행할 수 있습니다. \n     - 예시: 도커 설치, PostgreSQL 실행, 카프카 실행, 애플리케이션 실행, 카프카 서브스크라이버 실행 등.\n\n이 가이드는 Docker와 카프카 관련 문제 해결을 위한 기본적인 정보를 제공합니다. 자세한 내용이나 추가적인 도움이 필요할 경우 개발사나 전문가에게咨询하는 것이 좋습니다.'),
 Document(id='b2b06738-a00b-45b4-9bd4-33e65563cb71', metadata={}, page_content='# BIFE-Cycler 설치/

In [111]:
# RAG 체인 정의
rag_chain = (
    # 검색 결과를 포맷팅하고 질문을 처리합니다.
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 적용합니다.
    | model  # 모델을 적용합니다.
    | StrOutputParser()  # 문자열 출력 파서를 적용합니다.
)


In [113]:
rag_chain.invoke("bife-cycler 설치하는 방법을 순서대로 알려줘.")

'Docker 컨테이너가 이미 사용 중인 경우, 해당 이름을 재사용하기 위해선 컨테이너를 제거하거나 이름을 변경해야 합니다.\n\n1. **컨테이너 제거 방법**:\n   ```bash\ndocker rm [컨테이너 이름]\n```\n   예시: `docker rm pg-1`\n\n2. **웹 브라우저 접속 오류 발생 시**:\n   - 로그 확인 명령어:\n     ```bash\ndocker logs -f -n 1000 [인스턴스 이름]\n```\n     (예: server1, server2의 signet-server, signet-system에 대해 각각 로그 캡처 필요)\n\n3. **데이터가 들어오지 않는 경우**:\n   - `kafka-subscriber`의 로그에서 데이터 수신 확인\n   ```bash\ndocker logs -f -n 1000 kafka-subscriber\n```\n   - `kafka-subscriber`가 실행 중인지 확인\n   ```bash\ndocker ps -a | grep kafka-subscriber\n```\n   - 재시동 필요할 경우:\n     ```bash\ndocker restart kafka-subscriber\n```\n\n서버별 실행 스크립트는 다음과 같이 구성됩니다.\n\n### Server 1 (~/installation/)\n- `1.prerequisite`: 도커 설치 스크립트 (`install-docker.sh`)\n- `2.postgresql`: PostgreSQL 실행 관련 스크립트 (`run-as-active.sh` 및 `run-as-standby.sh`)\n- `3.kafka`: 카프카 실행 스크립트 (`run-kafka.sh`)\n- `4.application`: 애플리케이션 실행 및 버전 체크 관련 스크립트 (`run-server.sh`, `check-version-client.sh`, `check-version-server.sh`, `check-version-system.

In [130]:
for token in rag_chain.stream("bife-cycler 설치하는 방법을 순서대로 알려줘. 서버정보를 먼저 보여줘. 내용은 아주 상세하게 markdown으로 한글로 작성해줘. "):
    # print(token.content, end="")
    print(token, end="")

docker container 가 이미 사용중인 경우의 해결 방법과 서버별 실행 스크립트에 대한 정보를 제공했습니다. 도커 컨테이너 이름 충돌 시 제거하는 방법과, 로그 확인 및 데이터 들어오지 않는 경우의 처리 방안을 설명하였습니다. 또한 각 서버에서 실행해야 할 스크립트에 대한 구조와 파일 목록을 제공하였습니다.

In [126]:
for token in rag_chain.stream("bife-cycler에서 kafka를 설정하는 방법을 알려줘. 내용은 아주 상세하게 markdown으로 한글로 작성해줘. "):
    # print(token.content, end="")
    print(token, end="")

It appears you've provided a detailed guide on troubleshooting and running Docker containers for a specific application, likely related to Bife Cycler. The steps include checking if a container is already in use, removing it if necessary, and then proceeding with the execution of various scripts tailored for different servers (Server 1, Server 2, and Server 3) within the `~/installation/` directory. Each server has its set of scripts designed to install Docker, run PostgreSQL, Kafka, and other application components.

To address potential issues or questions you might have based on this information:

1. **Container Name Conflict**: If you encounter an error due to a container name conflict (e.g., `pg-1` is already in use), you can remove the existing container using `docker rm pg-1`. Then, you should be able to run your Docker command again without the conflict.

2. **Web Browser Access Issues**: For troubleshooting web browser access problems, checking the logs of the relevant contain